In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.17.1"

Helper method to read .env file

In [ ]:
using System.IO;

void LoadEnv()
{
    string filename = ".env";

    if (!File.Exists(filename))
        return;

    foreach (var line in File.ReadAllLines(filename))
    {
        var parts = line.Split(
            '=',
            StringSplitOptions.RemoveEmptyEntries);

        if (parts.Length != 2)
            continue;

        Environment.SetEnvironmentVariable(parts[0], parts[1]);
    }
}

Now we create the kernel

In [ ]:
// Import packages
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

LoadEnv();

var deployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_CHAT_DEPLOYMENT_MODEL");
var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_KEY");
var apiEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");

var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion(deployment,apiEndpoint,apiKey)                   
                   .Build();

var chat = kernel.GetRequiredService<IChatCompletionService>();


Simple question to OpenAI

In [ ]:
var result = await chat.GetChatMessageContentAsync("Hello, my name if John");

Console.WriteLine(result);

Now let's create a simple plugin in C#

In [ ]:
using System.Text.Json.Serialization;
using System.ComponentModel;

// First let's create the model

public class LightModel
{
   [JsonPropertyName("id")]
   public int Id { get; set; }

   [JsonPropertyName("name")]
   public string Name { get; set; }

   [JsonPropertyName("is_on")]
   public bool? IsOn { get; set; }
}

// Now the plugin of the light

public class LightsPlugin
{
   // Mock data for the lights
   private readonly List<LightModel> lights = new()
   {
      new LightModel { Id = 1, Name = "Table Lamp", IsOn = false },
      new LightModel { Id = 2, Name = "Porch light", IsOn = false },
      new LightModel { Id = 3, Name = "Chandelier", IsOn = true }
   };

   [KernelFunction("get_lights")]
   [Description("Gets a list of lights and their current state")]
   [return: Description("An array of lights")]
   public async Task<List<LightModel>> GetLightsAsync()
   {
      return lights;
   }

   [KernelFunction("change_state")]
   [Description("Changes the state of the light")]
   [return: Description("The updated state of the light; will return null if the light does not exist")]
   public async Task<LightModel?> ChangeStateAsync(int id, bool isOn)
   {
      var light = lights.FirstOrDefault(light => light.Id == id);

      if (light == null)
      {
         return null;
      }

      // Update the light with the new state
      light.IsOn = isOn;

      return light;
   }
}

Let's add the plugin to the kernel

In [4]:
// Add the plugin to the kernel

kernel.Plugins.AddFromType<LightsPlugin>("Lights");

// Enable planning
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

All good chat need a chat history

In [2]:
// Create a history store the conversation
var history = new ChatHistory();

Now let's get some light status and pass it to OpenAI for the answer using the power of SK

In [ ]:
history.AddUserMessage("Hello, what lights are on?");

var response = await chat.GetChatMessageContentAsync(history,
                                                     executionSettings: openAIPromptExecutionSettings,
                                                     kernel: kernel);